<div style="text-align: center;">
    <strong style="font-size: 2em;">FLEX Pipeline Overview</strong>
</div>

FLEX is a pipeline created to produce the Fourier - Laguerre coefficients $\hat{c}_{mn}$ and $\hat{s}_{mn}$ as shown in Equation 8 above. The pipeline relies on the user to input the galaxy ID's of disc galaxies they intend to search. Currently the pipeline is set up to search through the CEERS fields and the coressponing HST fields released in tandem (https://ceers.github.io/releases.html). 

The below diagram summarises the main framework of FLEX. 

![something](./Framework_Tree.png)


Below is a detailed description of how FLEX works.

<div style="text-align: left;">
    <strong style="font-size: 1em;">GalaxyDataExtractor class</strong>
</div>

1. detect_encoding: Read through data catalogue where RA and Dec values are defined to detect encoding
2. get_ra_dec: Search through catalogue and extract RA and Dec value based on user inputted galaxy IDs
3. sersic_profile: Sersic Profile the disc galaxy's luminosity distribution based on extracted pixel data
4. determine_galaxy_radius: Determine the entered galaxy ID's main radius beyond which background interference is removed
5. get_fits_data: Read in the respective filter's FITS image and create a 110 by 110 pixel cutout around the galactic centre. Then determine the galaxy radius using 'determine_galaxy_radius' method in the longest wavelength filter here being F444W. Based on this, mask out any values 
6. create_full_image: Create an image grid that comprises of four arrays stacked with each other: cropped image array and associated uncertainty and meshgrid of x and y pixels 
7. process_galaxies: This method performs several tasks as laid out below:
- Retrieving Right Ascension (RA) and Declination (Dec) coordinates from the CANDELS catalogue.
- Identifying the corresponding FITS file for each galaxy, based on its RA and Dec coordinates.
- Converting RA and Dec to pixel coordinates within the identified FITS file and checking if the galaxy falls within the image bounds.
- Using the F444W filter data to determine the radius of the galaxy in the FITS images.
- Extracting the galaxy image data and uncertainty arrays from multiple filters, centered on the galaxy and limited to the determined radius.
- Organizing the extracted data into a structured HDF5 file, where the data is categorized by filter.
- Logging any galaxies where the image data contains zero values or falls at the edge of the detector.

<div style="text-align: left;">
    <strong style="font-size: 1em;">LaguerreAmplitudes class</strong>
</div>

1. redsnapshot: Extract image array from HDF5 file created using GalaxyDataExtractor class
2. _gamma_n: Calculate the normalisation constant for the Laguerre Polynomial
3. _G_n: Compute the Laguerre Basis Function as shown in Equation 6 from Math_Overview.ipynb
4. laguerre_amplitudes: Compute laguerre sine and cos coefficients
5. calculate_A1: Define A1 metric that is utilised during centre finding process
6. calculate_gradient: At a certain (x,y) position calculate gradient of A1 matrix
7. find_centre: Determine centre of expansion by minimising A1 using Newton-Raphson's method 
8. find_center_moments: Calculate the center of mass (centroid) of the image's surface mass distribution
9. update_orders: After finding centre and scale length, adjust the mmax and nmax values as desired to account for finer structures. 
10. laguerre_reconstruction: Reconstruct the disc galaxy's surface density using the coefficients produced
11. compute_ratio: Calculate ratio metric for each rscl value

**Refer to A. Ganapathy et al (2024) for more details regarding centre finding and scale length optimisation**